In [1]:
import undetected_chromedriver as webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
import numpy as np
import pymysql

In [ ]:
# brighton_postcodes = ['BN1', 'BN13', 'BN17', 'BN21', 'BN25', 'BN41', 'BN45', 'BN8', 'BN10', 'BN14', 'BN18', 'BN22',
#              'BN26', 'BN42', 'BN5', 'BN9', 'BN11', 'BN15', 'BN2', 'BN23', 'BN27', 'BN43', 'BN6', 'BN12',
#              'BN16', 'BN20', 'BN24', 'BN3', 'BN44', 'BN7']

london_postcodes = pd.read_csv('./Data/London postcode districts.csv', header=0)
all_postcodes = london_postcodes['Postcode district'].tolist()
gotten_postcodes = pd.read_csv('./Data/gotten_postcodes_london.csv')['postcode'].tolist()
gotten_postcodes

postcodes = []
for post in all_postcodes:
    if post not in gotten_postcodes:
        postcodes.append(post)
print(len(postcodes))
postcodes

In [8]:
conn = pymysql.connect(
            host="localhost",          # Your host name
            user="root",      # Your username
            password="admin",  # Your password
            database="property_info_db"   # Your database name
        )
cur = conn.cursor()

In [9]:
def save_property_links_to_db(data, db_name):
    """
    Saves postcode and propert link to database
    """
    cur.execute(f"""INSERT INTO {db_name} (
        postcode, url) VALUES (%s, %s)""", (
        data['postcode'],
        data['url']
    ))
    
    conn.commit()
    print("Saved data to db successfully")

In [ ]:
def get_property_links(postcode):
    """
    retrieves the property links from the zoopla website
    :param postcodes: a list containing string representation of a postcode district
    :return: None
    """
#     urls_list = []
#     df = pd.DataFrame()
    data = {
        'url': "",
        'postcode': ""
    }
    
    try:
        s = Service("C:\\Users\\user\\Downloads\\Drivers\\chromedriver_win32\\chromedriver.exe")
        url = 'https://www.zoopla.co.uk/'
        
        chrome_options = Options()
        chrome_options.add_argument('--headless')
        chrome_options.add_argument('--disable-gpu') 

        driver = webdriver.Chrome(service=s, options=chrome_options)
        driver.get(url)

        # Maximize the window
        driver.maximize_window()

        time.sleep(5)
        consent_id = "gdpr-consent-notice"

        iframe = driver.find_element(By.ID, consent_id)

        # Switch to the iframe
        driver.switch_to.frame(iframe)

        # Click the accept button
        time.sleep(2)
        accept_cookies_btn = driver.find_element(By.XPATH, "//button[@id='save']")
        accept_cookies_btn.click()

        # switch back to main content
        driver.switch_to.default_content()

        # Find the search bar
        time.sleep(2)
        search = driver.find_element(By.XPATH, "//input[contains(@id,'downshift')]")
        search.send_keys(postcode)

        # Click the search button
        time.sleep(2)
        driver.find_element(By.XPATH, "//button[@class='x8jo560 x8jo562 x8jo56a _16fktr8']").click()

        # Saving data
        time.sleep(5)
        # get total number of properties in postcode
        total_properties = (
            int(driver.find_element(By.XPATH, '//p[@data-testid="total-results"]').text.strip('results').strip('')))
        print(f'There are {total_properties} properties in {postcode}')

        # # page index
        count = 0
        number_of_pages = (total_properties // 25) + 1
        i = 1

        while i <= number_of_pages:
            listings = driver.find_elements(By.XPATH, '//a[@class="_1maljyt1"]')
#             print(f'The length is {len(listings)}')
            print(f'Scrapping page {i} of {number_of_pages} from postcode {postcode}')
            for property_link in listings:
                count += 1
                link = property_link.get_attribute('href')
                data['postcode'] = postcode
                data['url'] = link or ""
                save_property_links_to_db(data, "london_properties_links")
                data['postcode'] = ""
                data['url'] = ""
                print(f"link {count}: {link}")
            i += 1
            url = f'https://www.zoopla.co.uk/for-sale/property/{postcode}/?q={postcode}&search_source=home&pn={i}'
            driver.get(url)
            time.sleep(3)
        print(f'Scraping completed for {postcode}')
    except ElementClickInterceptedException:
        print(f'code failing for postcode {postcode} due to element being intercepted')
    except TimeoutException:
        print(f'code failing for postcode {postcode} due to timeout')
    except NoSuchElementException:
        print(f'code failing for postcode {postcode} due to element not being found')
    finally:
        driver.quit()

In [ ]:
for postcode in postcodes:
    try:
        get_property_links(postcode)
    except Exception as ex:
        print(f"Error occurred : {ex}")
        continue
print('Done with all postcodes')

In [ ]:
bad_postcodes_list = []
postcodes = pd.read_csv('./Data/bad_postcodes.csv', index_col=[0])
# print(postcodes.head())
df_existing = pd.read_csv('./Data/property_urls.csv', index_col=[0])
# print(df_existing.head())

for postcode in postcodes['bad_postcodes'].values:
    try:
        df_postcode = get_property_links(postcode)
        df_existing = pd.concat([df_existing, df_postcode], ignore_index=True)
    except ElementClickInterceptedException:
        continue
    except TimeoutException:
        continue
    except NoSuchElementException:
        continue
    finally:
        df_existing.to_csv('./Data/property_urls_updated.csv')
        pd.DataFrame({'bad_postcodes':bad_postcodes_list}).to_csv('./Data/bad_postcodes_2.csv')
print('Done with all postcodes')

In [2]:
def get_property_info(property_link, postcode) -> dict:
    """
    Retrieves the information of a property
    :param property_link: a string representing the property page
    :param postcode: a string representing the property's postcode
    :return: data_dict: A dictionary containing the property data
    """
    data_dict = {
        "Price": "",
        "Address": "",
        "House Type": "",
        "Number of Bedrooms": "",
        "Number of Bathrooms": "",
        "Number of Receptions": "",
        "Other Features": "",
        "Tenure": "",
        "Lease Time": "",
        "Service Charge": "",
        "Tax Band": "",
        "Ground Rent": "",
        "Commonhold Details": "",
        "Points of Interest": "",
        "Listing Features": "",
        "Description": "",
        "Property Link": property_link,
        "Postcode": postcode
    }
    
    try:
        s = Service("C:\\Users\\user\\Downloads\\Drivers\\chromedriver_win32\\chromedriver.exe")
        chrome_options = Options()
        chrome_options.add_argument('--headless')
        chrome_options.add_argument('--disable-gpu') 

        driver = webdriver.Chrome(service=s, options=chrome_options)
        driver.get(property_link)

        # Maximize the window
        # time.sleep(2)
        driver.maximize_window()

        time.sleep(5)
        consent_id = "gdpr-consent-notice"

        iframe = driver.find_element(By.ID, consent_id)

        # Switch to the iframe
        driver.switch_to.frame(iframe)

        # Click the accept button
        time.sleep(2)
        accept_cookies_btn = driver.find_element(By.XPATH, "//button[@id='save']")
        accept_cookies_btn.click()

        # switch back to main content
        driver.switch_to.default_content()

        # time.sleep(2)

        # get price
        data_dict["Price"] = driver.find_element(By.XPATH, '//p[@data-testid="price"]').text or np.nan

        # address
        data_dict["Address"] = driver.find_element(By.XPATH, '//address[@data-testid="address-label"]').text or np.nan

        # house type
        data_dict["House Type"] = (driver.find_element(By.XPATH, '//p[@data-testid="title-label"]').text.lower()
                      .replace('for sale', "").strip() or np.nan)

        # features
        for feature in driver.find_elements(By.XPATH, '//div[@class="_1fuu7p80 _1fuu7p85 _1dgm2fc8 "]'):
            # Number of Bedrooms
            if 'bed' in feature.text.lower():
                data_dict["Number of Bedrooms"] = feature.text or np.nan
            # Number of Bathrooms
            elif 'bath' in feature.text.lower(): 
                data_dict["Number of Bathrooms"] = feature.text or np.nan
            # Number of Receptions
            elif 'reception' in feature.text.lower():
                data_dict["Number of Receptions"] = feature.text or np.nan
            # Other Features
            else:
                data_dict["Other Features"] = data_dict.get("Other Features", "") + str(feature.text) + ','
        data_dict["Other Features"] = data_dict.get("Other Features", "").rstrip(',')

        # contract info
        for info in driver.find_elements(By.XPATH, '//div[@class="_1k66bqh0"]'):
            # Tenure
            if 'tenure' in info.text.lower():
                data_dict["Tenure"] = info.find_element(By.XPATH, "following-sibling::*[1]").text or np.nan
            # Least Time
            elif 'lease' in info.text.lower():
                data_dict["Lease Time"] = info.find_element(By.XPATH, 'following-sibling::*[1]').text or np.nan
            # Service Charge
            elif 'service' in info.text.lower():
                data_dict["Service Charge"] = info.find_element(By.XPATH, 'following-sibling::*[1]').text or np.nan
            # Tax Band
            elif 'tax' in info.text.lower():
                data_dict["Tax Band"] = info.find_element(By.XPATH, 'following-sibling::*[1]').text or np.nan
            # Ground Rent
            elif 'rent' in info.text.lower():
                data_dict["Ground Rent"] = info.find_element(By.XPATH, 'following-sibling::*[1]').text or np.nan
            # Commonhold
            elif 'commonhold' in info.text.lower():
                data_dict["Commonhold Details"] = info.find_element(By.XPATH, 'following-sibling::*[1]').text or np.nan

        # points of interest
        for interest in driver.find_elements(By.XPATH, '//li[@class="cx0jbd2"]'):
            data_dict["Points of Interest"] = data_dict.get("Points of Interest", "") + str(interest.text) + ','
        data_dict["Points of Interest"] = data_dict.get("Points of Interest", "").rstrip(',')

        # listing features
        for listing_feature in driver.find_elements(By.XPATH, '//li[@class="swbww71"]'):
            data_dict["Listing Features"] = data_dict.get("Listing Features", "") + listing_feature.text + '\n'

        # description
        data_dict["Description"] = driver.find_element(By.XPATH, '//div[@class="ru2q7m3"]').text or np.nan
        
    except Exception as ex:
        print(f'An error occurred for link in get_data_func: {property_link}: {ex}')
#         bad_link = {'Link':property_link, 'postcode': postcode}
#         bad_links.append(bad_link)
    finally:
        driver.quit()
        time.sleep(2)
    return data_dict

In [3]:
def save_to_db(data):
    """
    Saves extracted data to an sql database
    :param data
    """
    cur.execute("""INSERT INTO london_properties (
        price, address, house_type, number_of_bedrooms, number_of_bathrooms, number_of_receptions, other_features, tenure, 
        lease_time, service_charge, tax_band, ground_rent, commonhold_details, points_of_interest, listing_features, 
        description_text, property_link, postcode) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, (
    data['Price'],
    data['Address'],
    data['House Type'],
    data['Number of Bedrooms'],
    data['Number of Bathrooms'],
    data['Number of Receptions'],
    data['Other Features'],
    data['Tenure'],
    data['Lease Time'],
    data['Service Charge'],
    data['Tax Band'],
    data['Ground Rent'],
    data['Commonhold Details'],
    data['Points of Interest'],
    data['Listing Features'],
    data['Description'],
    data['Property Link'],
    data['Postcode']))
    
    conn.commit()
    print("Saved to db successfully")

In [5]:
property_data = pd.read_csv('./Data/property_urls_london.csv', index_col = [0], skiprows=46524, header=None)
property_data.columns = ['postcode', 'url']
lk = "https://www.zoopla.co.uk/for-sale/details/64126913/?search_identifier=cacfdbd9fe6366ad29a28485d34c9c4c0520caad8004665eb6a0abd73328948d"
property_data[property_data['url'] == lk]
# property_data

,postcode,url
0,,


In [ ]:
# List of property links and corresponding postcodes
property_data = pd.read_csv('./Data/property_urls_london.csv', index_col = [0], skiprows=46524, header=None)
property_data.columns = ['postcode', 'url']
# Initialize a list to store the scraped data
# data_list = []

# Bad links
# bad_links = []

# counter
count = 46523

# Loop through each property link and postcode
for row in property_data.itertuples():
    count += 1
    try:
        print(f"Starting extraction for link {count}: {row.url}")
        data_dict = get_property_info(row.url, row.postcode)
        save_to_db(data_dict)
        print(f"Done with link {count}: {row.url}")
#         break
    except Exception as ex:
        print(f'An error occurred for link {count} {row.url}: {ex}')
        bad_link = {'Link':row.url, 'postcode': row.postcode}
#         bad_links.append(bad_link)
        continue

print("Completed Data Extraction")

could not detect version_main.therefore, we are assuming it is chrome 108 or higher


Starting extraction for link 46524: https://www.zoopla.co.uk/for-sale/details/64122658/?search_identifier=cacfdbd9fe6366ad29a28485d34c9c4c0520caad8004665eb6a0abd73328948d


could not detect version_main.therefore, we are assuming it is chrome 108 or higher


Saved to db successfully
Done with link 46524: https://www.zoopla.co.uk/for-sale/details/64122658/?search_identifier=cacfdbd9fe6366ad29a28485d34c9c4c0520caad8004665eb6a0abd73328948d
Starting extraction for link 46525: https://www.zoopla.co.uk/for-sale/details/64119059/?search_identifier=cacfdbd9fe6366ad29a28485d34c9c4c0520caad8004665eb6a0abd73328948d


could not detect version_main.therefore, we are assuming it is chrome 108 or higher


...

Saved to db successfully
Done with link 51384: https://www.zoopla.co.uk/for-sale/details/63323262/?search_identifier=49448d5167a8364ab8be241234f371ee9027d1dedd9206a0fd5045c6f0bd5c5f
Starting extraction for link 51385: https://www.zoopla.co.uk/for-sale/details/63323263/?search_identifier=49448d5167a8364ab8be241234f371ee9027d1dedd9206a0fd5045c6f0bd5c5f


could not detect version_main.therefore, we are assuming it is chrome 108 or higher


Saved to db successfully
Done with link 51385: https://www.zoopla.co.uk/for-sale/details/63323263/?search_identifier=49448d5167a8364ab8be241234f371ee9027d1dedd9206a0fd5045c6f0bd5c5f
Starting extraction for link 51386: https://www.zoopla.co.uk/for-sale/details/63323244/?search_identifier=49448d5167a8364ab8be241234f371ee9027d1dedd9206a0fd5045c6f0bd5c5f


could not detect version_main.therefore, we are assuming it is chrome 108 or higher


Saved to db successfully
Done with link 51386: https://www.zoopla.co.uk/for-sale/details/63323244/?search_identifier=49448d5167a8364ab8be241234f371ee9027d1dedd9206a0fd5045c6f0bd5c5f
Starting extraction for link 51387: https://www.zoopla.co.uk/for-sale/details/62586786/?search_identifier=0aed7e4dc0ed00bccdec7994bb7c594989ec40ccb58dc704f7c21b011911086d


In [ ]:
bad_links

In [ ]:
current_links = pd.read_csv('./Data/extracted_data_brighon.csv', usecols=[0])
all_the_links = pd.read_csv('./Data/property_urls_updated.csv', index_col = [0])

In [ ]:
all_links = all_the_links['urls'].tolist()
links_extracted = current_links['property_link'].tolist()

links_to_get_list =  []

for link_to_get in all_links:
    if link_to_get not in links_extracted:
        links_to_get_list.append(True)
    else:
        links_to_get_list.append(False)
links_to_get_list
len(links_to_get_list)
links_to_get = all_the_links.loc[links_to_get_list]
links_to_get

In [ ]:
# counter
count = 0

# Loop through each property link and postcode
for row in links_to_get.itertuples():
    count += 1
    try:
        data_dict = get_property_info(row.urls, row.postcode)
        save_to_db(data_dict)   
    except Exception as ex:
        print(f'An error occurred for link {count} {row.urls}: {ex}')
        continue
    finally:
        print(f"Done with link {count}: {row.urls}")

print("Completed Data Extraction")